In [2]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob


In [3]:
IMAGE_SIZE = [224, 224]

In [4]:
train_directory="D:/ANIMALS50/ANIMALS50/50/Friendly"
test_directory="D:/ANIMALS50/ANIMALS50/50/Friendly"
val_directory="D:/ANIMALS50/ANIMALS50/50/Friendly"

In [5]:
# add preprocessing layer to the front of VGG
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

# don't train existing weights
for layer in vgg.layers:
  layer.trainable = False

58889256/58889256 [==============================] - 27s 0us/step


In [6]:
  # useful for getting number of classes
folders = glob("D:/ANIMALS50/ANIMALS50/50/Friendly/*")
len(folders)

105

In [7]:
# our layers - you can add more if you want
x = Flatten()(vgg.output)
# x = Dense(1000, activation='relu')(x)
prediction = Dense(len(folders), activation='softmax')(x)

In [8]:
model = Model(inputs=vgg.input, outputs=prediction)

# view the structure of the model
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [9]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [10]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory(train_directory,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(test_directory,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 4682 images belonging to 105 classes.
Found 4682 images belonging to 105 classes.


In [11]:
print(len(training_set))
print(len(test_set))

147
147


In [12]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

C:\Users\ashne\AppData\Local\Temp\ipykernel_416\675562961.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/5
147/147 [==============================] - 555s 4s/step - loss: 4.2123 - accuracy: 0.2450 - val_loss: 1.6948 - val_accuracy: 0.5880
Epoch 2/5
147/147 [==============================] - 530s 4s/step - loss: 1.6208 - accuracy: 0.6117 - val_loss: 0.9644 - val_accuracy: 0.7604
Epoch 3/5
147/147 [==============================] - 537s 4s/step - loss: 1.1199 - accuracy: 0.7279 - val_loss: 0.5834 - val_accuracy: 0.8618
Epoch 4/5
147/147 [==============================] - 537s 4s/step - loss: 0.7884 - accuracy: 0.7986 - val_loss: 0.4778 - val_accuracy: 0.8665
Epoch 5/5
147/147 [==============================] - 532s 4s/step - loss: 0.5729 - accuracy: 0.8475 - val_loss: 0.2196 - val_accuracy: 0.9351


In [13]:
model.save('abc.h5')

In [3]:
from keras.models import load_model
from tensorflow.keras.utils import load_img,img_to_array

model1 = load_model('./Model/abc.h5',compile=False) 

In [4]:
import numpy as np
def output(location):
    img=load_img(location,target_size=(224,224,3))
    img=img_to_array(img)
    img=img/255
    img=np.expand_dims(img,[0])
    answer=model1.predict(img)
    abcd=np.max(answer, axis=1)
    print(abcd)

img="D:/websites/species/images to predict/1.jpg"
pic=load_img(img,target_size=(224,224,3))
output(img)

1/1 [==============================] - 0s 428ms/step
[0.99768627]
